In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
# X_train = dataset[0][0][0]['AABA'].values
# meanX = X_train.mean(axis=1)
# stdX = X_train.std(axis = 1)
# X_train = X_train.sub(meanX, axis=0)
# X_train = X_train.div(stdX, axis = 0)
# X_train = X_train.values

# X_train = dataset[0][0][0].values
# X_train = (X_train - X_train.mean())/X_train.std()
# y_train = dataset[0][0][1]['AABA'].values

# returns = dataset[1][0][0]
# returns = (returns - returns.mean()) / returns.std()
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [7]:
X_train = dataset[0][0][0]
meanX = X_train.mean(axis=1)
stdX = X_train.std(axis = 1)
X_train = X_train.sub(meanX, axis=0)
X_train = X_train.div(stdX, axis = 0)
X_train = X_train.values

y_train = dataset[0][0][1].values

X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [8]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])
y_train = get_one_hot(y_train, 2)
y_test = get_one_hot(y_test, 2)

In [9]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31)
y test shape: (490, 31, 2)


In [10]:
timestep = 10

In [11]:
data = X_train
targets = y_train

train_gen = TimeseriesGenerator(data, targets,
                               length=timestep, sampling_rate=1,
                               batch_size=740)

In [12]:
data = X_test
targets = y_test

test_gen = TimeseriesGenerator(data, targets,
                               length=timestep, sampling_rate=1,
                               batch_size=250)

In [13]:
X_train = train_gen[0][0]
y_train = train_gen[0][1]
X_test = test_gen[0][0]
y_test = test_gen[0][1]

In [14]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (740, 10, 31)
y train shape: (740, 31, 2)
x test shape: (250, 10, 31)
y test shape: (250, 31, 2)


In [15]:
X_train = X_train.transpose((0,2,1))
X_train = np.reshape(X_train, (740 * 31, timestep))
y_train = np.reshape(y_train, (740 * 31, 2))

In [16]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [17]:
print(f"x train shape: {X_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {X_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (22940, 10, 1)
y train shape: (22940, 2)
x test shape: (250, 10, 31)
y test shape: (250, 31, 2)


In [20]:
# expected input data shape: (batch_size, timesteps, data_dim)

# The LSTM architecture
regressor = Sequential()
# regressor.add(LSTM(units=25, return_sequences=True, input_shape=(240, 31)))
regressor.add(LSTM(units=25, input_shape=(timestep, 1), dropout=0.1))


# regressor.add(LSTM(units = 10, input_shape = (X_train.shape[1], 1)))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units = 50, return_sequences = True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units = 50, return_sequences = True))
# regressor.add(Dropout(0.2))

# regressor.add(LSTM(units = 10))
# regressor.add(Dropout(0.2))


regressor.add(Dense(2, activation='softmax'))

regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
# regressor.compile(loss='mean_squared_error',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=1000,batch_size=128, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=100),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

Train on 18352 samples, validate on 4588 samples
Epoch 1/1000
18352/18352 [==============================] - 2s 112us/step - loss: 0.6928 - acc: 0.5134 - val_loss: 0.6932 - val_acc: 0.5153
Epoch 2/1000
18352/18352 [==============================] - 2s 84us/step - loss: 0.6927 - acc: 0.5154 - val_loss: 0.6927 - val_acc: 0.5129
Epoch 3/1000
18352/18352 [==============================] - 2s 82us/step - loss: 0.6926 - acc: 0.5166 - val_loss: 0.6928 - val_acc: 0.5163
Epoch 4/1000
18352/18352 [==============================] - 2s 83us/step - loss: 0.6925 - acc: 0.5181 - val_loss: 0.6929 - val_acc: 0.5126
Epoch 5/1000
18352/18352 [==============================] - 2s 84us/step - loss: 0.6926 - acc: 0.5172 - val_loss: 0.6929 - val_acc: 0.5102
Epoch 6/1000
18352/18352 [==============================] - 2s 84us/step - loss: 0.6926 - acc: 0.5155 - val_loss: 0.6929 - val_acc: 0.5133
Epoch 7/1000
18352/18352 [==============================] - 2s 83us/step - loss: 0.6924 - acc: 0.5145 - val_loss: 0.

18352/18352 [==============================] - 2s 91us/step - loss: 0.6914 - acc: 0.5227 - val_loss: 0.6943 - val_acc: 0.5146
Epoch 60/1000
18352/18352 [==============================] - 2s 87us/step - loss: 0.6914 - acc: 0.5237 - val_loss: 0.6946 - val_acc: 0.5007
Epoch 61/1000
18352/18352 [==============================] - 2s 88us/step - loss: 0.6911 - acc: 0.5266 - val_loss: 0.6945 - val_acc: 0.5122
Epoch 62/1000
18352/18352 [==============================] - 2s 96us/step - loss: 0.6914 - acc: 0.5227 - val_loss: 0.6942 - val_acc: 0.5161
Epoch 63/1000
18352/18352 [==============================] - 2s 88us/step - loss: 0.6912 - acc: 0.5247 - val_loss: 0.6951 - val_acc: 0.5163
Epoch 64/1000
18352/18352 [==============================] - 2s 90us/step - loss: 0.6914 - acc: 0.5218 - val_loss: 0.6945 - val_acc: 0.5133
Epoch 65/1000
18352/18352 [==============================] - 2s 89us/step - loss: 0.6912 - acc: 0.5237 - val_loss: 0.6947 - val_acc: 0.5133
Epoch 66/1000
18352/18352 [=======

18352/18352 [==============================] - 2s 87us/step - loss: 0.6887 - acc: 0.5272 - val_loss: 0.6966 - val_acc: 0.5085
Epoch 118/1000
18352/18352 [==============================] - 2s 83us/step - loss: 0.6892 - acc: 0.5269 - val_loss: 0.6958 - val_acc: 0.5078


In [ ]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [ ]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int

In [ ]:
(sum(y_test[:, :, 1] == label[:, :, 1])/(y_test.size/2)).sum()

In [ ]:
model_name = '../model/LSTM/my_model3.h5'
regressor.save(model_name)  # creates a HDF5 file 'my_model.h5'
del regressor  # deletes the existing model

In [ ]:
from keras.models import load_model
from keras.activations import softmax

# returns a compiled model
# identical to the previous one
regressor1 = load_model(model_name, custom_objects={"softmax": softmax})

In [ ]:
predicted_load = regressor1.predict(X_test)
label_load = predicted_load > 0.5
label_load = label_load * 1 # Convert boolean to int
(sum(y_test[:, :, 1] == label_load[:, :, 1])/(y_test.size/2)).sum()